# OLAP

Nesta faze vamos transformar a nossa base de dados para o formato estrela definido.

# 0.1 Funções Auxiliares

In [2]:
def excuteSingleSQLstatement(sql, host, database, user, password):
    conn = pg.connect(host=host,database=database, user=user, password=password)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close()    

In [3]:
def pandafy(rows, colnames):
    N=len(colnames)
    D={cn: [] for cn in colnames}
    for row in rows:
        for i in range(N): D[colnames[i]].append(row[i])
    pdfy=pd.DataFrame(D)
    return pdfy

# 0.2 Imports

In [4]:
import psycopg2 as pg
import pandas as pd

# 1. Main Code

### Create StarSchema tables

In [4]:
SQL_CREATE_STAR_SCHEMA = """ 
CREATE TABLE dim_dates
(
    date_id smallint NOT NULL,
    year smallint NOT NULL,
    month smallint NOT NULL,
    CONSTRAINT dates_pkey PRIMARY KEY (date_id)
);

CREATE TABLE dim_boroughs
(
    id integer NOT NULL,
    area_name character varying(100) COLLATE pg_catalog."default" NOT NULL,
    inner_outer_london integer NOT NULL,
    gla_pop_estimate_2016 double precision NOT NULL,
    gla_household_estimate_2016 double precision NOT NULL,
    inland_area_hectares double precision NOT NULL,
    pop_density_per_hectare_2016 double precision NOT NULL,
    average_age_2016 double precision NOT NULL,
    ppt_pop_aged_0_15_2016 double precision NOT NULL,
    ppt_pop_wrkg_age_2016 double precision NOT NULL,
    ppt_pop_aged_65_and_over_2016 double precision NOT NULL,
    net_internal_migration_2014 double precision NOT NULL,
    net_international_migration_2014 double precision NOT NULL,
    net_natural_change_2014 double precision NOT NULL,
    pct_resident_pop_born_abroad_2014 double precision NOT NULL,
    largest_migrant_pop_by_country_birth_2011 character varying(100) COLLATE pg_catalog."default" NOT NULL,
    pct_largest_migrant_pop_2011 double precision NOT NULL,
    second_largest_migrant_pop_by_country_birth_2011 character varying(100) COLLATE pg_catalog."default" NOT NULL,
    pct_second_largest_migrant_pop_2011 double precision NOT NULL,
    third_largest_migrant_pop_by_country_birth_2011 character varying(100) COLLATE pg_catalog."default" NOT NULL,
    pct_third_largest_migrant_pop_2011 double precision NOT NULL,
    pct_pop_from_bame_groups_2016 double precision NOT NULL,
    pct_people_aged_3_plus_main_is_not_english_2011_census double precision NOT NULL,
    overseas_nationals_entering_the_uk_nino_2014_15 double precision NOT NULL,
    new_migrant_nino_rates_2014_15 double precision NOT NULL,
    largest_migrant_pop_arrived_during_2014_15 character varying(100) COLLATE pg_catalog."default" NOT NULL,
    second_largest_migrant_pop_arrived_during_2014_15 character varying(100) COLLATE pg_catalog."default" NOT NULL,
    third_largest_migrant_pop_arrived_during_2014_15 character varying(100) COLLATE pg_catalog."default" NOT NULL,
    employment_rate_pct_2015 double precision NOT NULL,
    male_employment_rate_2015 double precision NOT NULL,
    female_employment_rate_2015 double precision NOT NULL,
    unemployment_rate_2015 double precision NOT NULL,
    youth_unemployment_claimant_rate_18_24_dec_14 double precision NOT NULL,
    ppt_16_18_year_olds_are_neet_pct_2014 double precision NOT NULL,
    ppt_the_wrkg_age_pop_claim_out_work_benefits_pct_aug_2015 double precision NOT NULL,
    pct_wrkg_age_with_a_disability_2015 double precision NOT NULL,
    ppt_wrkg_age_people_with_no_qualifications_pct_2015 double precision NOT NULL,
    ppt_wrkg_age_with_degree_or_equi_and_above_pct_2015 double precision NOT NULL,
    gross_annual_pay_2015 double precision NOT NULL,
    gross_annual_pay_male_2015 double precision NOT NULL,
    gross_annual_pay_female_2015 double precision NOT NULL,
    modelled_household_median_income_estimates_2012_13 double precision NOT NULL,
    pct_adults_that_vltd_in_past_12_months_2010_11_to_2012_13 double precision NOT NULL,
    number_jobs_by_workplace_2014 double precision NOT NULL,
    pct_employment_that_is_in_public_sector_2014 double precision NOT NULL,
    jobs_density_2014 double precision NOT NULL,
    number_active_businesses_2014 double precision NOT NULL,
    two_year_business_survival_rates_started_in_2012 double precision NOT NULL,
    crime_rates_per_thousand_pop_2014_15 double precision NOT NULL,
    fires_per_thousand_pop_2014 double precision NOT NULL,
    ambulance_incidents_per_hundred_pop_2014 double precision NOT NULL,
    median_house_price_2014 double precision NOT NULL,
    average_band_d_council_tax_charge_pound_2015_16 double precision NOT NULL,
    new_homes_net_2014_15_provisional double precision NOT NULL,
    homes_owned_outright_2014_pct double precision NOT NULL,
    being_bought_with_mortgage_or_loan_2014_pct double precision NOT NULL,
    rented_from_local_authority_or_housing_assoc_2014_pct double precision NOT NULL,
    rented_from_private_landlord_2014_pct double precision NOT NULL,
    pct_area_that_is_greenspace_2005 double precision NOT NULL,
    total_carbon_emissions_2013 double precision NOT NULL,
    household_waste_recycling_rate_2014_15 double precision NOT NULL,
    number_cars_2011_census double precision NOT NULL,
    number_cars_per_household_2011_census double precision NOT NULL,
    pct_adults_cycle_at_least_once_per_month_2013_14 double precision NOT NULL,
    average_public_transport_accessibility_score_2014 double precision NOT NULL,
    achv_5_more_a_c_grds_gcse_or_sim_icld_eng_n_maths_2013_14 double precision NOT NULL,
    rates_children_looked_after_2015 double precision NOT NULL,
    pct_pupils_first_is_not_english_2015 double precision NOT NULL,
    pct_children_living_in_out_work_households_2014 double precision NOT NULL,
    male_life_expectancy_2012_14 double precision NOT NULL,
    female_life_expectancy_2012_14 double precision NOT NULL,
    teenage_conception_rate_2014 double precision NOT NULL,
    life_satisfaction_score_2011_14_out_10 double precision NOT NULL,
    worthwhileness_score_2011_14_out_10 double precision NOT NULL,
    happiness_score_2011_14_out_10 double precision NOT NULL,
    anxiety_score_2011_14_out_10 double precision NOT NULL,
    childhood_obesity_prevalance_pct_2014_15 double precision NOT NULL,
    people_aged_17_plus_with_diabetes_pct double precision NOT NULL,
    mortality_rate_from_causes_considered_preventable_2012_14 double precision NOT NULL,
    political_control_in_council character varying(100) COLLATE pg_catalog."default" NOT NULL,
    ppt_seats_won_by_conservatives_in_2014_election double precision NOT NULL,
    ppt_seats_won_by_labour_in_2014_election double precision NOT NULL,
    ppt_seats_won_by_lib_dems_in_2014_election double precision NOT NULL,
    turnout_at_2014_local_elections double precision NOT NULL,
    CONSTRAINT demographics_pkey PRIMARY KEY (id)
);

CREATE TABLE dim_category
(
    crime_id smallint NOT NULL,
    major_category_id character varying(200) NOT NULL,
    minor_category_id character varying(200) NOT NULL,
    CONSTRAINT crimes_pkey PRIMARY KEY (crime_id)
);

CREATE TABLE fact_crime
(
    ocurred_id integer NOT NULL,
    crime_id smallint NOT NULL,
    date_id smallint NOT NULL,
    borough_id smallint NOT NULL,
    value smallint NOT NULL,
    CONSTRAINT ocurred_pkey PRIMARY KEY (ocurred_id),
    CONSTRAINT ocurred_borough_id FOREIGN KEY (borough_id)
        REFERENCES public.dim_boroughs (id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
        NOT VALID,
    CONSTRAINT ocurred_crime_id FOREIGN KEY (crime_id)
        REFERENCES public.dim_category (crime_id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
        NOT VALID,
    CONSTRAINT ocurred_date_id FOREIGN KEY (date_id)
        REFERENCES public.dim_dates (date_id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
        NOT VALID
);

CREATE TABLE dim_criminals
(
    id integer NOT NULL,
    gender character(1) COLLATE pg_catalog."default" NOT NULL,
    age_range character varying(20) NOT NULL,
    self_defined_ethnicity character varying(200) NOT NULL,
    officer_defined_ethnicity character varying(200) NOT NULL,
    CONSTRAINT criminal_pkey PRIMARY KEY (id)
);
"""

In [34]:
# Criar dates

SQL = """CREATE TABLE fact_crime
(
    ocurred_id integer NOT NULL,
    crime_id smallint NOT NULL,
    date_id smallint NOT NULL,
    borough_id smallint NOT NULL,
    value smallint NOT NULL,
    CONSTRAINT ocurred_pkey PRIMARY KEY (ocurred_id),
    CONSTRAINT ocurred_borough_id FOREIGN KEY (borough_id)
        REFERENCES public.dim_boroughs (id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
        NOT VALID,
    CONSTRAINT ocurred_crime_id FOREIGN KEY (crime_id)
        REFERENCES public.dim_category (crime_id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
        NOT VALID,
    CONSTRAINT ocurred_date_id FOREIGN KEY (date_id)
        REFERENCES public.dim_dates (date_id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
        NOT VALID
);"""

host = 'localhost'
database = 'star_crimes_db'
user="postgres"
password="1234"

excuteSingleSQLstatement(SQL, host, database, user, password)

In [30]:
host = 'localhost'
database = 'star_crimes_db'
user="postgres"
password="1234"

excuteSingleSQLstatement(SQL_CREATE_STAR_SCHEMA, host, database, user, password)

NameError: name 'SQL_CREATE_STAR_SCHEMA' is not defined

# 2. Load Tables

### Load dim_dates

In [9]:
SQL_crime_db = """
SELECT * FROM public.dates
"""

In [10]:
conn = pg.connect(host="localhost",database="crimes_db", user="postgres", password="1234")
cur = conn.cursor()
cur.execute(SQL_crime_db)
emps = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
cur.close()
conn.close()

df=pandafy(emps, colnames)
df

,date_id,year,month
0,0,2013,7
1,1,2015,5
2,2,2013,8
3,3,2011,10
4,4,2016,4
...,...,...,...
123,123,2018,4
124,124,2018,5
125,125,2018,6
126,126,2018,7


In [11]:
df.to_csv('C:/csvs/dates.csv', index=False)  

In [12]:
SQL_LOAD = """
COPY dim_dates (date_id,year,month) 
FROM 'C:/csvs/dates.csv'
DELIMITER ','
CSV HEADER;
"""

In [13]:
host = 'localhost'
database = 'star_crimes_db'
user="postgres"
password="1234"
excuteSingleSQLstatement(SQL_LOAD, host, database, user, password)

### Load dim_boroughs

In [22]:
SQL_crime_db = """
SELECT * FROM public.demographics
"""

In [24]:
conn = pg.connect(host="localhost",database="crimes_db", user="postgres", password="1234")
cur = conn.cursor()
cur.execute(SQL_crime_db)
emps = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
cur.close()
conn.close()

df=pandafy(emps, colnames)
df.head()

,id,area_name,inner_outer_london,gla_pop_estimate_2016,gla_household_estimate_2016,inland_area_hectares,pop_density_per_hectare_2016,average_age_2016,ppt_pop_aged_0_15_2016,ppt_pop_wrkg_age_2016,...,happiness_score_2011_14_out_10,anxiety_score_2011_14_out_10,childhood_obesity_prevalance_pct_2014_15,people_aged_17_plus_with_diabetes_pct,mortality_rate_from_causes_considered_preventable_2012_14,political_control_in_council,ppt_seats_won_by_conservatives_in_2014_election,ppt_seats_won_by_labour_in_2014_election,ppt_seats_won_by_lib_dems_in_2014_election,turnout_at_2014_local_elections
0,0,Barking and Dagenham,0,205773.0,76841.0,3610.8,57.3,32.9,21.0,86.1,...,7.05,3.05,25.3,7.3,227.6,Lab,0.0,100.0,0.0,36.5
1,1,Barnet,0,385108.0,149147.0,8674.8,44.5,37.2,21.0,83.3,...,7.37,2.75,18.4,6.0,133.8,Cons,50.8,47.6,1.6,40.5
2,2,Bexley,0,243303.0,97233.0,6058.1,39.9,38.9,20.8,89.0,...,7.21,3.29,21.4,6.9,164.3,Cons,71.4,23.8,0.0,39.6
3,3,Brent,0,328568.0,119166.0,4323.3,76.1,35.5,20.1,82.5,...,7.22,2.92,23.9,7.9,169.4,Lab,9.5,88.9,1.6,36.3
4,4,Bromley,0,326560.0,139654.0,15013.5,21.7,40.1,15.8,88.4,...,7.44,3.26,16.5,5.2,148.5,Cons,85.0,11.7,0.0,40.8


In [25]:
df.to_csv('C:/csvs/boroughs.csv', index=False)  

In [39]:
SQL_LOAD = """
COPY dim_boroughs (
id,
area_name,
inner_outer_london,
gla_pop_estimate_2016,
gla_household_estimate_2016,
inland_area_hectares,
pop_density_per_hectare_2016,
average_age_2016,
ppt_pop_aged_0_15_2016,
ppt_pop_wrkg_age_2016,
ppt_pop_aged_65_and_over_2016,
net_internal_migration_2014,
net_international_migration_2014,
net_natural_change_2014,
pct_resident_pop_born_abroad_2014,
largest_migrant_pop_by_country_birth_2011,
pct_largest_migrant_pop_2011,
second_largest_migrant_pop_by_country_birth_2011,
pct_second_largest_migrant_pop_2011,
third_largest_migrant_pop_by_country_birth_2011,
pct_third_largest_migrant_pop_2011,
pct_pop_from_bame_groups_2016,
pct_people_aged_3_plus_main_is_not_english_2011_census,
overseas_nationals_entering_the_uk_nino_2014_15,
new_migrant_nino_rates_2014_15,
largest_migrant_pop_arrived_during_2014_15,
second_largest_migrant_pop_arrived_during_2014_15,
third_largest_migrant_pop_arrived_during_2014_15,
employment_rate_pct_2015,
male_employment_rate_2015,
female_employment_rate_2015,
unemployment_rate_2015,
youth_unemployment_claimant_rate_18_24_dec_14,
ppt_16_18_year_olds_are_neet_pct_2014,
ppt_the_wrkg_age_pop_claim_out_work_benefits_pct_aug_2015,
pct_wrkg_age_with_a_disability_2015,
ppt_wrkg_age_people_with_no_qualifications_pct_2015,
ppt_wrkg_age_with_degree_or_equi_and_above_pct_2015,
gross_annual_pay_2015,
gross_annual_pay_male_2015,
gross_annual_pay_female_2015,
modelled_household_median_income_estimates_2012_13,
pct_adults_that_vltd_in_past_12_months_2010_11_to_2012_13,
number_jobs_by_workplace_2014,
pct_employment_that_is_in_public_sector_2014,
jobs_density_2014,
number_active_businesses_2014,
two_year_business_survival_rates_started_in_2012,
crime_rates_per_thousand_pop_2014_15,
fires_per_thousand_pop_2014,
ambulance_incidents_per_hundred_pop_2014,
median_house_price_2014,
average_band_d_council_tax_charge_pound_2015_16,
new_homes_net_2014_15_provisional,
homes_owned_outright_2014_pct,
being_bought_with_mortgage_or_loan_2014_pct,
rented_from_local_authority_or_housing_assoc_2014_pct,
rented_from_private_landlord_2014_pct,
pct_area_that_is_greenspace_2005,
total_carbon_emissions_2013,
household_waste_recycling_rate_2014_15,
number_cars_2011_census,
number_cars_per_household_2011_census,
pct_adults_cycle_at_least_once_per_month_2013_14,
average_public_transport_accessibility_score_2014,
achv_5_more_a_c_grds_gcse_or_sim_icld_eng_n_maths_2013_14,
rates_children_looked_after_2015,
pct_pupils_first_is_not_english_2015,
pct_children_living_in_out_work_households_2014,
male_life_expectancy_2012_14,
female_life_expectancy_2012_14,
teenage_conception_rate_2014,
life_satisfaction_score_2011_14_out_10,
worthwhileness_score_2011_14_out_10,
happiness_score_2011_14_out_10,
anxiety_score_2011_14_out_10,
childhood_obesity_prevalance_pct_2014_15,
people_aged_17_plus_with_diabetes_pct,
mortality_rate_from_causes_considered_preventable_2012_14,
political_control_in_council,
ppt_seats_won_by_conservatives_in_2014_election,
ppt_seats_won_by_labour_in_2014_election,
ppt_seats_won_by_lib_dems_in_2014_election,
turnout_at_2014_local_elections
) 
FROM 'C:/csvs/boroughs.csv'
DELIMITER ','
CSV HEADER;
"""

In [40]:
host = 'localhost'
database = 'star_crimes_db'
user="postgres"
password="1234"
excuteSingleSQLstatement(SQL_LOAD, host, database, user, password)

### Load Categorys

In [41]:
SQL_crime_db = """
SELECT c.crime_id, a.major_category, b.minor_category
FROM public.major_categorys a, public.minor_categorys b, public.crimes c
WHERE c.major_category_id = a.major_category_id AND c.major_category_id = b.minor_category_id
"""

In [44]:
conn = pg.connect(host="localhost",database="crimes_db", user="postgres", password="1234")
cur = conn.cursor()
cur.execute(SQL_crime_db)
emps = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
cur.close()
conn.close()

df=pandafy(emps, colnames)
df.head()

,crime_id,major_category,minor_category
0,0,Violence Against the Person,Harassment
1,1,Theft and Handling,Other Theft
2,2,Violence Against the Person,Harassment
3,3,Violence Against the Person,Harassment
4,4,Violence Against the Person,Harassment


In [45]:
df.to_csv('C:/csvs/categorys.csv', index=False)  

In [48]:
SQL_LOAD = """
COPY dim_category (
crime_id,major_category_id,minor_category_id
) 
FROM 'C:/csvs/categorys.csv'
DELIMITER ','
CSV HEADER;
"""

In [49]:
host = 'localhost'
database = 'star_crimes_db'
user="postgres"
password="1234"
excuteSingleSQLstatement(SQL_LOAD, host, database, user, password)

### Load Fact Crime

In [23]:
SQL_crime_db = """ 
SELECT crime_id, date_id, borough_id, sum(value)
FROM public.ocurred
GROUP BY crime_id, date_id, borough_id
"""

In [24]:
conn = pg.connect(host="localhost",database="crimes_db", user="postgres", password="1234")
cur = conn.cursor()
cur.execute(SQL_crime_db)
emps = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
cur.close()
conn.close()

df=pandafy(emps, colnames)
df.head()

,crime_id,date_id,borough_id,sum
0,6,109,7,466
1,6,126,7,575
2,8,83,7,921
3,4,49,7,165
4,22,7,7,13


In [25]:
df['ocurred_id'] = df.index

In [26]:
df

,crime_id,date_id,borough_id,sum,ocurred_id
0,6,109,7,466,0
1,6,126,7,575,1
2,8,83,7,921,2
3,4,49,7,165,3
4,22,7,7,13,4
...,...,...,...,...,...
3726,14,4,7,23,3726
3727,38,121,7,12,3727
3728,1,109,7,8096,3728
3729,22,79,7,2,3729


In [18]:
df.to_csv('C:/csvs/crimes.csv', index=False)  

In [35]:
SQL_LOAD = """
COPY fact_crime (
crime_id,date_id, borough_id, value,ocurred_id
) 
FROM 'C:/csvs/fact_table_small.csv'
DELIMITER ','
CSV HEADER;
"""

In [36]:
host = 'localhost'
database = 'star_crimes_db'
user="postgres"
password="1234"
excuteSingleSQLstatement(SQL_LOAD, host, database, user, password)